In [7]:
# TF-IDF 기반 영화 추천 (콘텐츠 유사도: 줄거리+장르)
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches

# 1) 샘플 데이터셋 (제목, 장르, 줄거리)
data = [
    {"title":"기생충", "genres":"드라마 스릴러", "overview":"가난한 가족이 부유한 집에 취업하며 벌어지는 블랙코미디 스릴러"},
    {"title":"옥자", "genres":"드라마 모험", "overview":"거대한 동물 옥자와 소녀의 우정을 지키기 위한 모험"},
    {"title":"부산행", "genres":"액션 스릴러 좀비", "overview":"좀비가 창궐한 열차에서 벌어지는 생존기"},
    {"title":"더 웨일", "genres":"드라마", "overview":"세상과 단절된 채 살던 교수가 딸과의 관계를 회복하려 한다"},
    {"title":"인터스텔라", "genres":"SF 드라마 모험", "overview":"인류의 미래를 위해 우주로 떠나는 아버지와 동료들의 여정"},
    {"title":"그래비티", "genres":"SF 스릴러", "overview":"우주에서 고립된 우주비행사들의 생존기"},
    {"title":"매드맥스 분노의 도로", "genres":"액션 모험", "overview":"사막의 종말 세계에서 카체이스와 해방을 향한 싸움"},
    {"title":"인셉션", "genres":"SF 스릴러 액션", "overview":"타인의 꿈 속에 들어가 아이디어를 주입하는 팀의 작전"},
    {"title":"라라랜드", "genres":"드라마 로맨스 음악", "overview":"배우 지망생과 재즈 피아니스트의 사랑과 꿈"},
    {"title":"위플래쉬", "genres":"드라마 음악", "overview":"천재 드러머 지망생과 혹독한 스승의 집착과 성장"},
    {"title":"코코", "genres":"애니메이션 가족 음악", "overview":"음악을 사랑하는 소년이 죽은 자의 나라에서 가족의 비밀을 찾는다"},
    {"title":"토이 스토리", "genres":"애니메이션 가족 모험", "overview":"장난감들의 우정과 모험이 펼쳐지는 따뜻한 이야기"},
    {"title":"가디언즈 오브 갤럭시", "genres":"SF 액션 코미디", "overview":"우주 악당들로부터 은하계를 지키는 유쾌한 히어로 팀"},
    {"title":"닥터 스트레인지", "genres":"SF 액션 판타지", "overview":"교만한 천재 외과의가 마법사가 되어 세상을 위협으로부터 지킨다"},
    {"title":"셰이프 오브 워터", "genres":"드라마 판타지 로맨스", "overview":"말 못 하는 여성과 수중 생명체 사이의 특별한 사랑"}
]
df = pd.DataFrame(data)

# 2) 전처리: 추천에 쓸 텍스트 컬럼 만들기(장르+줄거리)
df["text"] = ((df["genres"].fillna("")) * 2 + " " + df["overview"].fillna("")).str.lower()

# 3) TF-IDF 벡터화
#   - ngram_range=(1,2): 단어와 바이그램 반영
#   - min_df=1: 샘플이 적어서 1로 두지만, 실제는 2~5 추천
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
tfidf = vectorizer.fit_transform(df["text"])

# 4) 코사인 유사도 행렬
cosine_sim = cosine_similarity(tfidf, tfidf)

# 5) 제목 → 인덱스 매핑
title_to_idx = {t.lower(): i for i, t in enumerate(df["title"])}

def _resolve_title(user_title: str):
    """사용자 입력 제목을 데이터셋 제목과 유사 매칭."""
    key = user_title.strip().lower()
    if key in title_to_idx:
        return key
    # 근사 매칭(오타/부분 입력 대응)
    candidates = get_close_matches(key, list(title_to_idx.keys()), n=1, cutoff=0.5)
    return candidates[0] if candidates else None

def recommend(title: str, top_n: int = 5, include_score: bool = True):
    """주어진 제목과 유사한 영화 TOP-N 추천"""
    key = _resolve_title(title)
    if not key:
        print(f"[경고] '{title}' 제목을 찾지 못했습니다. 데이터셋 제목 중 하나를 입력하세요.")
        return pd.DataFrame()
    idx = title_to_idx[key]
    sim_scores = list(enumerate(cosine_sim[idx]))
    # 자기 자신 제외 + 유사도 내림차순
    sim_scores = sorted([(i,s) for i,s in sim_scores if i != idx], key=lambda x: -x[1], reverse=False)[:top_n]
    rec_idx = [i for i,_ in sim_scores]
    result = df.loc[rec_idx, ["title","genres","overview"]].copy()
    if include_score:
        result["similarity"] = [round(s, 4) for _, s in sim_scores]
    return result.reset_index(drop=True)

# 6) 사용 예시
print("=== [기생충] 비슷한 영화 (없으면 근사매칭) ===")
display(recommend("기생충", top_n=5))

print("=== [인셉션] 비슷한 영화 ===")
display(recommend("인셉션", top_n=5))

print("=== [라라랜드] 비슷한 영화 ===")
display(recommend("라라랜드", top_n=5))


=== [기생충] 비슷한 영화 (없으면 근사매칭) ===


,title,genres,overview,similarity
0,부산행,액션 스릴러 좀비,좀비가 창궐한 열차에서 벌어지는 생존기,0.1439
1,인셉션,SF 스릴러 액션,타인의 꿈 속에 들어가 아이디어를 주입하는 팀의 작전,0.0943
2,그래비티,SF 스릴러,우주에서 고립된 우주비행사들의 생존기,0.0645
3,인터스텔라,SF 드라마 모험,인류의 미래를 위해 우주로 떠나는 아버지와 동료들의 여정,0.0325
4,위플래쉬,드라마 음악,천재 드러머 지망생과 혹독한 스승의 집착과 성장,0.0189


=== [인셉션] 비슷한 영화 ===


,title,genres,overview,similarity
0,부산행,액션 스릴러 좀비,좀비가 창궐한 열차에서 벌어지는 생존기,0.1209
1,기생충,드라마 스릴러,가난한 가족이 부유한 집에 취업하며 벌어지는 블랙코미디 스릴러,0.0943
2,그래비티,SF 스릴러,우주에서 고립된 우주비행사들의 생존기,0.0878
3,가디언즈 오브 갤럭시,SF 액션 코미디,우주 악당들로부터 은하계를 지키는 유쾌한 히어로 팀,0.0597
4,닥터 스트레인지,SF 액션 판타지,교만한 천재 외과의가 마법사가 되어 세상을 위협으로부터 지킨다,0.0550


=== [라라랜드] 비슷한 영화 ===


,title,genres,overview,similarity
0,위플래쉬,드라마 음악,천재 드러머 지망생과 혹독한 스승의 집착과 성장,0.1380
1,셰이프 오브 워터,드라마 판타지 로맨스,말 못 하는 여성과 수중 생명체 사이의 특별한 사랑,0.0900
2,인터스텔라,SF 드라마 모험,인류의 미래를 위해 우주로 떠나는 아버지와 동료들의 여정,0.0339
3,코코,애니메이션 가족 음악,음악을 사랑하는 소년이 죽은 자의 나라에서 가족의 비밀을 찾는다,0.0268
4,기생충,드라마 스릴러,가난한 가족이 부유한 집에 취업하며 벌어지는 블랙코미디 스릴러,0.0183
